In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
from torch import nn
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer
from transformers import BertModel
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import f1_score
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import torch.nn.functional as F
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
from util import *
import torch.nn.functional as F
from datetime import datetime
import pprint
import os
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
import util
#from catalyst.metrics.functional import process_multilabel_components
#from catalyst.metrics import multi_label_accuracy
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.core.decorators import auto_move_data
from sklearn.metrics import label_ranking_average_precision_score, accuracy_score, f1_score
#from ignite.utils import convert_tensor
#from pytorch_metric_learning.miners import MultiSimilarityMiner
#from pytorch_metric_learning.losses import TripletMarginLoss
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    )

In [3]:
# training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
# testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

# subset = list(sorted(set(all_tiers_100)-set(["PersonalizedProduct"])))

# nice_subset = [tier_translations[x] for x in subset]

# #model_name = "/home/martin/IdeaProjects/phenetics/bertForPatents/"
# model_name = "allenai/scibert_scivocab_uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# #base_model = AutoModel.from_pretrained(model_name, gradient_checkpointing=True)

# import util

# from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder



# training_labels = training_set[subset].apply(util.array_labels_textual, axis=1)
# testing_labels = testing_set[subset].apply(util.array_labels_textual, axis=1)
# all_labels = np.concatenate((training_labels, testing_labels))
# enc = LabelEncoder()
# enc.fit(all_labels)
# training_set['labels'] = enc.transform(training_labels).astype(np.int64).tolist()
# testing_set['labels'] = enc.transform(testing_labels).astype(np.int64).tolist()



# training_set['labels']

# mlb = MultiLabelBinarizer()

# #training_set['labels']=
# training_labels = mlb.fit_transform(training_set[subset].apply(util.array_labels, axis=1)).astype(np.float32)
# testing_labels = mlb.transform(testing_set[subset].apply(util.array_labels, axis=1)).astype(np.float32)
# #testing_set['labels']=testing_set[subset].astype(int).values.tolist()

In [4]:
# training_set['labels'] = training_labels.tolist()
# testing_set['labels'] = testing_labels.tolist()

In [5]:
class PatentDataset(Dataset):

    
    def __init__(self, dataframe, tokenizer, abstract_max_len=160, claims_max_len=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.claims_max_len = claims_max_len
        self.abstract_max_len = abstract_max_len

        self.claims = dataframe.claims
        self.abstracts = dataframe.abstract        
        self.labels = dataframe.labels

    def tokenize(self, text, max_len):
        text = str(text)
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            f"input_ids": torch.tensor(ids, dtype=torch.long),
            f"attention_mask": torch.tensor(mask, dtype=torch.long),
            f"token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        }
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        
        abstract = self.tokenize(self.abstracts[index], max_len=self.abstract_max_len)
        claims = self.tokenize(self.claims[index], max_len=self.claims_max_len)
        labels = torch.tensor(self.labels[index])
        return {"abstract": abstract, 
                "claims": claims,
                'labels': labels}
    


In [6]:
def stack(listOfDicts):
    initDict = listOfDicts[0]
    finalDict = {}
    for key in initDict.keys():
        tensors = tuple(d[key] for d in listOfDicts)
        finalDict[key] = torch.stack(tensors)
    return finalDict

In [7]:
set(tier1) & set(all_tiers_100)

{'AnalysisAndModeling',
 'AnatomicalTarget',
 'Imaging',
 'Manufacturing',
 'SpecificationofUse',
 'SurgicalMethod'}

In [8]:
from sklearn.metrics import classification_report, f1_score

In [9]:
from pytorch_lightning.metrics import functional as FM


class BasicSystem(pl.LightningModule):
    def __init__(self, batch_size=16, learning_rate = 1e-5):
        super().__init__()
        
        #self.model_name = "allenai/scibert_scivocab_uncased"
        self.model_name = "bertForPatents/"
    
        filtered = set(all_tiers_100)-set(["PersonalizedProduct"])
        self.subset = list(sorted(set(tier1) & set(filtered)))
        #self.subset = list(sorted(set(all_tiers_100)-set(["PersonalizedProduct"])))16
        
        
        self.loss_function = torch.nn.BCEWithLogitsLoss()
        
        #torch.nn.MSELoss() #torch.nn.MultiLabelSoftMarginLoss()  #torch.nn.BCEWithLogitsLoss() #torch.nn.MultiLabelSoftMarginLoss() #torch.nn.CrossEntropyLoss()
        #self.softmax = torch.nn.Softmax(dim=1)
        
        self.train_accuracy = pl.metrics.Accuracy()
        self.val_accuracy = pl.metrics.Accuracy()
        
        self.train_accuracy = pl.metrics.Accuracy()
        self.train_f1 = pl.metrics.F1()
        
        self.val_accuracy = pl.metrics.Accuracy()
        self.val_f1 = pl.metrics.F1()

        self.learning_rate = learning_rate
        self.batch_size = batch_size
        logdir="/var/patentmark/logdir/fit2/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        
    def prepare_data(self):
        self.training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
        self.testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        
    def setup(self,stage):

        training_labels = self.training_set[self.subset].apply(util.array_labels, axis=1)
        testing_labels = self.testing_set[self.subset].apply(util.array_labels, axis=1)
        all_labels = np.concatenate((training_labels, testing_labels))
        
        self.label_encoder = MultiLabelBinarizer()
        self.label_encoder.fit(all_labels)
        print(f"Labels: {self.label_encoder.classes_}")
        
        self.training_set['labels'] = self.label_encoder.transform(training_labels).astype(np.float).tolist()
        self.testing_set['labels'] = self.label_encoder.transform(testing_labels).astype(np.float).tolist()
        
        self.training_dataset = PatentDataset(self.training_set, self.tokenizer)
        self.testing_dataset = PatentDataset(self.testing_set, self.tokenizer)
        
        self.text_embedder = AutoModel.from_pretrained(self.model_name, gradient_checkpointing=True)
        
        total_embedding_size = self.text_embedder.pooler.dense.out_features*2
        output_size = len(self.label_encoder.classes_)
        bottleneck = 256
        
        self.dropout1 = nn.Dropout(0.1)
        self.bottleneck = nn.Linear(total_embedding_size, bottleneck)
        self.classifier = nn.Linear(bottleneck, output_size)

    
    def embed_patent(self, abstract, claims):
        abstract_emb = self.text_embedder(input_ids=abstract["input_ids"], attention_mask=abstract["attention_mask"])
        abstract_emb = abstract_emb[1]
        
        claim_emb = self.text_embedder(input_ids=claims["input_ids"], attention_mask=claims["attention_mask"])
        claim_emb = claim_emb[1]
    
        x = torch.cat((abstract_emb, claim_emb), 1)
        x = self.dropout1(x)
        x = self.bottleneck(x)
        x = F.elu(x)
        
        return x
    
    def classify_patent(self, embedding):
        x = embedding
        x = self.dropout1(x)
        x = self.classifier(x)
        return x
    
    @auto_move_data
    def forward(self, abstract, claims):
        x = self.embed_patent(abstract, claims)
        x = self.classify_patent(x)
        return x
        
    def train_dataloader(self):
        return DataLoader(self.training_dataset, batch_size=self.batch_size, shuffle=True, num_workers=32)

    def val_dataloader(self):
        return DataLoader(self.testing_dataset, batch_size=self.batch_size, shuffle=False, num_workers=32)
            
    def training_step(self, batch, batch_idx):
        abstract = batch['abstract']
        claims = batch['claims']
        labels = batch['labels']
        
        embedding = self.embed_patent(abstract, claims)
        logits = self.classify_patent(embedding)
        loss = self.loss_function(logits, labels)
        
        #acc = FM.accuracy(F.sigmoilogits, labels)
        #self.log("train_loss", loss, prog_bar=True, on_epoch=True, on_step=False)
        #self.log("train_acc", acc, prog_bar=True, on_epoch=True, on_step=False)
        
        #self.log("train_loss", loss, prog_bar=True)       
        
        #predictions = torch.argmax(logits, dim=1)
        #self.log("train_f1", f1_score(labels.detach().cpu(), predictions.detach().cpu(), average="weighted"), prog_bar=True, on_epoch=True, on_step=True)
        
        return loss
    
    
    def validation_step(self, batch, batch_idx):
        abstract = batch['abstract']
        claims = batch['claims']
        labels = batch['labels']
    
        embedding = self.embed_patent(abstract, claims)
        logits = self.classify_patent(embedding)
        loss = self.loss_function(logits, labels)
        self.log("val_loss", loss, prog_bar=True)
        

        #acc = FM.accuracy(F.sigmoid(logits), labels)
        #self.log("val_acc", acc, prog_bar=True, on_epoch=True, on_step=False)
        
        predictions = (F.sigmoid(logits).cpu().detach().numpy() >= 0.5).tolist()
        labels = labels.cpu().detach().numpy().astype(np.bool).tolist()
        
        return predictions, labels
    
    def validation_epoch_end(self, outputs):
        predictions = list(f.cat(x[0] for x in outputs))
        labels = list(f.cat(x[1] for x in outputs))
        print(classification_report(labels, predictions, target_names=self.label_encoder.classes_))
        

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
#         param_optimizer = list(self.named_parameters())
#         no_decay = ["bias", "gamma", "beta"]
#         optimizer_grouped_parameters = [
#                 {
#                     "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
#                     "weight_decay_rate": 0.01
#                     },
#                 {
#                     "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
#                     "weight_decay_rate": 0.0
#                     },
#                 ]
#         optimizer = torch.optim.AdamW(
#                 optimizer_grouped_parameters,
#                 lr=self.learning_rate,
#                 )
#         return optimizer
        

In [10]:
model = BasicSystem()
early_stopping = EarlyStopping('val_loss')
trainer = pl.Trainer(gpus=1, accumulate_grad_batches=1,
                     callbacks=[early_stopping],
                     precision=16,
                     auto_scale_batch_size=True,
                     #auto_lr_find=True,
                     log_every_n_steps=1, 
                     flush_logs_every_n_steps=1)


GPU available: True, used: True
2020-12-05 14:52:01 - GPU available: True, used: True
TPU available: False, using: 0 TPU cores
2020-12-05 14:52:01 - TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2020-12-05 14:52:01 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
2020-12-05 14:52:01 - Using native 16bit precision.


In [11]:
#trainer.tune(model)

In [12]:
trainer.fit(model)

Labels: ['Analysis and Modeling' 'Anatomical Target' 'Imaging' 'Manufacturing'
 'Specification of Use' 'Surgical Method']



  | Name           | Type              | Params
-----------------------------------------------------
0 | loss_function  | BCEWithLogitsLoss | 0     
1 | train_accuracy | Accuracy          | 0     
2 | val_accuracy   | Accuracy          | 0     
3 | train_f1       | F1                | 0     
4 | val_f1         | F1                | 0     
5 | text_embedder  | BertModel         | 344 M 
6 | dropout1       | Dropout           | 0     
7 | bottleneck     | Linear            | 524 K 
8 | classifier     | Linear            | 1.5 K 
2020-12-05 14:52:17 - 
  | Name           | Type              | Params
-----------------------------------------------------
0 | loss_function  | BCEWithLogitsLoss | 0     
1 | train_accuracy | Accuracy          | 0     
2 | val_accuracy   | Accuracy          | 0     
3 | train_f1       | F1                | 0     
4 | val_f1         | F1                | 0     
5 | text_embedder  | BertModel         | 344 M 
6 | dropout1       | Dropout           | 0     
7 | 

                       precision    recall  f1-score   support

Analysis and Modeling       0.44      1.00      0.61        14
    Anatomical Target       0.00      0.00      0.00        22
              Imaging       0.00      0.00      0.00        18
        Manufacturing       0.00      0.00      0.00        12
 Specification of Use       0.31      1.00      0.48        10
      Surgical Method       0.60      0.38      0.46         8

            micro avg       0.39      0.32      0.35        84
            macro avg       0.23      0.40      0.26        84
         weighted avg       0.17      0.32      0.20        84
          samples avg       0.40      0.31      0.33        84



                       precision    recall  f1-score   support

Analysis and Modeling       0.00      0.00      0.00        84
    Anatomical Target       0.67      1.00      0.81       164
              Imaging       0.58      0.96      0.72       133
        Manufacturing       0.00      0.00      0.00        83
 Specification of Use       0.00      0.00      0.00        79
      Surgical Method       0.00      0.00      0.00        40

            micro avg       0.63      0.50      0.56       583
            macro avg       0.21      0.33      0.25       583
         weighted avg       0.32      0.50      0.39       583
          samples avg       0.64      0.53      0.55       583



                       precision    recall  f1-score   support

Analysis and Modeling       0.00      0.00      0.00        84
    Anatomical Target       0.68      0.99      0.80       164
              Imaging       0.57      0.96      0.72       133
        Manufacturing       0.33      0.07      0.12        83
 Specification of Use       0.00      0.00      0.00        79
      Surgical Method       0.00      0.00      0.00        40

            micro avg       0.62      0.51      0.56       583
            macro avg       0.26      0.34      0.27       583
         weighted avg       0.37      0.51      0.41       583
          samples avg       0.62      0.54      0.54       583



                       precision    recall  f1-score   support

Analysis and Modeling       0.44      0.10      0.16        84
    Anatomical Target       0.71      0.93      0.80       164
              Imaging       0.58      0.83      0.68       133
        Manufacturing       0.37      0.34      0.35        83
 Specification of Use       0.00      0.00      0.00        79
      Surgical Method       1.00      0.17      0.30        40

            micro avg       0.60      0.52      0.56       583
            macro avg       0.52      0.39      0.38       583
         weighted avg       0.52      0.52      0.47       583
          samples avg       0.62      0.56      0.54       583



                       precision    recall  f1-score   support

Analysis and Modeling       0.00      0.00      0.00        84
    Anatomical Target       0.67      1.00      0.81       164
              Imaging       0.62      0.70      0.65       133
        Manufacturing       0.43      0.18      0.25        83
 Specification of Use       0.00      0.00      0.00        79
      Surgical Method       0.33      0.20      0.25        40

            micro avg       0.62      0.48      0.54       583
            macro avg       0.34      0.35      0.33       583
         weighted avg       0.41      0.48      0.43       583
          samples avg       0.65      0.52      0.54       583




1

In [13]:
# abstracts = stack(list(tokenize(tokenizer, x, max_len=MAX_LEN_ABSTRACT) for x in testing_set.abstract))

# claims = stack(list(tokenize(tokenizer, x, max_len=MAX_LEN_CLAIMS) for x in testing_set.claims))

# predictions = model.forward(abstract=abstracts, claims=claims)

# binarized = predictions.detach().numpy() > 0.5

# from sklearn.metrics import classification_report
# testing_labels = testing_set[subset]
# print(classification_report(testing_labels, binarized))